# [실습문제] LLM 프롬프트로 Anki 단어장 생성하기

In [4]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [5]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [11]:
def Akti_Voca_Note(before_text, model='gpt-4o-mini', temperature=1, top_p=1):
  # 역할/페르소나/지시사항
  system_message = """
  영어 텍스트를 입력하면, 중요한 어휘를 추출해 Anki 카드 형식의 단어장 데이터를 생성한다.

  # Steps
  - LLM이 영어 텍스트에서 학습 가치가 있는 주요 단어 또는 어구를 추출하여 단어장으로 만들어야 한다.
  - 너무 쉬운 단어는 제외하고, 고유명사, 전문용어, 관용구 등도 포함한다.
  - 각 카드에는 (1)단어/어구, (2)뜻(영어 또는 한글), (3)품사, (4)예문(영어)을 포함한다.
  - 각 카드는 Anki에서 바로 쓸 수 있는 포맷(각 항목은 세미콜론 ; 으로 구분, 한 줄에 한 카드)으로 출력한다.

  # Output Format
  입력 텍스트에서 추출된 단어장 데이터를 다음 형식으로 제공

  각 카드는 단어 또는 어구;뜻;품사;영어 예문
  모든 카드는 한 줄에 한 개, 세미콜론(;)으로 필드 구분
  불필요한 부가설명, 줄바꿈, 문장 없음

  - 입력: 입력 텍스트
  - 출력: 단어장 데이터

  # Examples
  - 입력: The committee will convene next week to discuss the new policy proposals and reach a consensus.
  - 출력: convene;to come together for a meeting;verb;The committee will convene next week.
          consensus;general agreement;noun;The group reached a consensus after a long discussion.
          proposal;a suggestion or plan;noun;He submitted a proposal for the new project.'

  # Extra Instructions

  """

  user_message = f"""
  다음 영어 텍스트에서 중요한 어휘를 추출해 Anki 카드 형식의 단어장 데이터를 생성

  {before_text}

  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

before_text = 'If you enter English text, extract important vocabulary, extract an important vocabulary that generates an important vocabulary in an Antki card format.'
output = Akti_Voca_Note(before_text)
print(output)


enter;to go or come into;verb;You need to enter your password to access the system.  
extract;to remove or obtain something from a source;verb;We need to extract the relevant data from the report.  
vocabulary;the body of words used in a particular language;noun;The vocabulary of the English language is vast and diverse.  
generate;to produce or create;verb;The software can generate reports automatically.  
Antki;not a recognized term;N/A;N/A  


In [17]:
import pandas as pd

lines = output.strip().split('\n')
parsed_data = [line.split(';') for line in lines]


columns = ['Word/Phrase	', 'Definition', 'PoS', 'Example Sentence']
df = pd.DataFrame(parsed_data, columns=columns)

# # 출력
# print(df)

[['enter',
  'to go or come into',
  'verb',
  'You need to enter your password to access the system.  '],
 ['extract',
  'to remove or obtain something from a source',
  'verb',
  'We need to extract the relevant data from the report.  '],
 ['vocabulary',
  'the body of words used in a particular language',
  'noun',
  'The vocabulary of the English language is vast and diverse.  '],
 ['generate',
  'to produce or create',
  'verb',
  'The software can generate reports automatically.  '],
 ['Antki', 'not a recognized term', 'N/A', 'N/A']]